# Selecting and Filtering Data

```{admonition} Summary
:class: hint


```

For analyzing the data, if the data is huge, the processing with Jupyter will take more time, or sometimes the result will not be visualized. So the data could be limited to small parts by selecting the data of interest.

Load data first.

In [154]:
import geopandas as gp
import pandas as pd
from pathlib import Path
INPUT = Path.cwd().parents[0] / "00_data"
gdb_path = INPUT / "Biotopwerte_Dresden_2018.gdb"
gdf = gp.read_file(gdb_path, layer="Biotopwerte_Dresden_2018")

The headers (attributes) of the data are called.

In [155]:
list(gdf.columns)

['CLC_st1', 'Biotpkt2018', 'Shape_Length', 'Shape_Area', 'geometry']

It is important to have information about the format of each attribute. 

`````{admonition} Essentials of Attribute Formatting
:class: note
When working with attributes:
- Use quotes ('  ') for text or numbers stored as string (object) format
- For numeric values (like integers or floats), use them directly without quotes.

`````

To check the format of the attributes the `dtypes` method is used.

In [156]:
gdf.dtypes

CLC_st1           object
Biotpkt2018      float64
Shape_Length     float64
Shape_Area       float64
geometry        geometry
dtype: object

```{figure} ../resources/22-2.png
:name: figure-example

Attributes in Dataset Based on the Metadata
````

## Unique Values Selection


The `unique` function returns all distinct values of an attribute.


**ex:** In this dataset, the `CLC_st1` attribute might have some identical values for land use (such as '313' for mixed forest or '512' for water bodies) but multiple features in the dataset recorded as mixed forest or water bodies.

```{figure} ../resources/19.png
:name: figure-example

Classification Code Interpretation
````

By defining the attribute of interest in this example `CLC_st1`, the unique values in this attribute are visualized.

In [157]:
unique_groups = gdf['CLC_st1'].unique()
unique_groups

array(['122', '124', '511', '332', '322', '112', '111', '121', '123',
       '142', '132', '211', '133', '231', '221', '222', '141', '324',
       '311', '312', '313', '321', '131', '333', '411', '512'],
      dtype=object)

Also the number of the unique values can be accessed by `nunique` function.

In [158]:
unique_numbers = gdf['CLC_st1'].nunique()
unique_numbers

26

`````{admonition} None values in nunique function!
:class: danger

By default, the `nunique` function does not count **null** or **missing** values.
`````

For including the `None` values as unique values, `dropna` parameter should modify to False.

In [159]:
unique_numbers = gdf['CLC_st1'].nunique(dropna = False)
unique_numbers

26

The frequency of each unique value can be counted using the `groupby` method.


The data then is grouped by unique values, and the number of entries in each group is calculated using the `size` function.

In the following example, just 4 features recorded with the code of "133" representing construction sites, while the code of "511" representing water cources has 2563 records in this dataset.

`````{admonition} Order of Groups with size function
:class: tip, dropdown
The result of `size` function is sorted based on unique values:
- For **string** unique values, sorting is in **alphabetical** order.
- For **number** unique values, sorting is in **ascending numerical** order.


**ex:** In the following example, the unique values are in the number format so they ordered ascending as 111, 112, 121, ...
`````


In [193]:
groupcounts = gdf.groupby('CLC_st1').size()
pd.DataFrame(groupcounts).T

CLC_st1,111,112,121,122,123,124,131,132,133,141,...,312,313,321,322,324,332,333,411,511,512
0,1115,7187,2732,5454,12,133,16,47,4,1627,...,1546,1594,8,667,525,36,10,3,2563,234


The frequency of unique values can also be counted using the `value_counts` method.


`````{admonition} Order of Groups with value_counts function
:class: tip, dropdown
The output of the `value_counts` function is sorted in descending order based on the frequency counts.


**ex:** In the following example the counts ordered descending.
`````


In [161]:
groupcounts = gdf["CLC_st1"].value_counts()
pd.DataFrame(groupcounts).T

CLC_st1,112,122,231,121,511,142,311,141,313,312,...,222,132,332,131,221,123,333,321,133,411
count,7187,5454,3293,2732,2563,2413,1787,1627,1594,1546,...,56,47,36,16,14,12,10,8,4,3


In a long list of unique values, if the number of features for a specific group is needed, it can be accessed like a value for a key in a dictionary.

`````{admonition} Format Importance Use Case
:class: warning,
The importance of attribute formatting, mentioned at the beginning of this chapter, will be highlighted here. In the following examples, the values for **both** attributes of "CLC_st1" and "Biotpkt2018" are **numbers**, while the format of the first is an object (string) and the second is a float. So for calling the "CLC_st1" values the quotes ('') are required, while for the "Biotpkt2018" values the quotes are not required. 
`````

- **String Format**

In the following example, the output shows that in this dataset, 2413 features recorded with code 142: Sport and leisure facilities

In [162]:
sv_count = groupcounts['142']
sv_count

2413

- **Float Format**

In [163]:
group_2 = gdf.groupby('Biotpkt2018').size()
group_2

Biotpkt2018
0.000000      16
0.500000      55
1.000000      52
1.500000     102
2.000000     131
            ... 
16.928546     36
17.079406      3
17.381302     10
18.055116    667
20.164493      8
Length: 69, dtype: int64

In [164]:
group_2[1.000000]

52

```{admonition} Importance of the precision of float format!
:class: note, dropdown
Sometimes, due to the precision of the float format, the printed output of the `groupby` or `value_counts` function may not exactly match the value in the DataFrame. In this case, if a value from their output is called directly, an error will occur.

The following figure is an example of this note, when the value "18.055116" called. This value is an output of the `groupby` function in the previous step, but it is a rounded value, and this value in the dataset includes additional decimal places like 18.05511611 or 18.05511689 or ...

```{figure} ../resources/14_.png
:width: 700px
:name: figure-example 
Error of calling rounded values instead of precise values in the dataset
```


## Selection of String Values

After becoming familiar with attributes and unique values of the features, the required features can be selected:

If only features with a specific value are needed, the value can be called directly:


In [165]:
filtered_data = gdf[gdf['CLC_st1'] == '133']
filtered_data.head(5)

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry
11831,133,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228..."
24343,133,6.035751,287.703701,4278.338534,"MULTIPOLYGON (((411656.913 5656302.92, 411664...."
24344,133,6.035751,183.544225,2066.507754,"MULTIPOLYGON (((411567.287 5656337.438, 411562..."
24347,133,6.035751,213.015432,2684.440542,"MULTIPOLYGON (((411587.489 5655959.618, 411588..."


## Selection of Float Values


In [166]:
filtered_data = gdf[gdf['Biotpkt2018'] == 1.000000]
filtered_data.head(5)

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry
9492,111,1.0,411.855290,2480.458787,"MULTIPOLYGON (((406985.474 5655597.694, 406984..."
9504,111,1.0,89.433492,466.944726,"MULTIPOLYGON (((416782.224 5652196.728, 416782..."
9514,111,1.0,172.927119,1621.632535,"MULTIPOLYGON (((419300.05 5657313.72, 419299.9..."
9515,111,1.0,320.578260,3691.824728,"MULTIPOLYGON (((408302.234 5655785.262, 408296..."
9519,111,1.0,121.899927,920.751286,"MULTIPOLYGON (((414687.929 5655817.21, 414656...."


`````{admonition} Importance of the precision of float format!
:class: note, dropdown
As mentioned in the previous note, for float values, due to precision, it may sometimes show an empty DataFrame (meaning no feature with this value exists), while in `groupby` or `value_counts` functions, it may indicate that some features with this value exist.

```{figure} ../resources/15_.png
:width: 600px
:name: figure-example
Empty output for calling rounded values instead of precise values in the dataset
`````

In [167]:
# Codes of the upper note
filtered_data = gdf[gdf['Biotpkt2018'] == 18.055116]
filtered_data.head(5)

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry


If features with more than one specific value are needed, the values can be called as a list using `isin` method.

In [168]:
filtered_data = gdf[gdf['CLC_st1'].isin(['133', '321', '411'])]
filtered_data.head(5)

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry
11831,133,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228..."
23859,321,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579..."
23860,321,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618...."
23861,321,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516..."
23862,321,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151..."


## Dimension of Dataset

To access the dimension of the dataset, meaning the number of rows and columns, the method `Shape` is used.

In the following example the dataset includes 15 rows (features) and 6 columns (attributes).

In [169]:
filtered_data.shape

(15, 5)

## Adding New Attributes (Columns)

To add a new column to a dataset, the column name is defined for the dataset, and values are assigned to each row.

In the following example, the output of the previous step is used as the dataset, and a new column named **test** is added. 

`````{admonition} [ : , 'column_name' ]
:class: attention, dropdown

Using `:` sign before comma `,` means selecting all the rows, and the information after comma `,` specifies the columns of interest.
`````

Since the dataset contains 15 rows, 15 values are assigned using the `loc` method. (The values could be from different formats: string, null, negative, float, integer, ...)

In [171]:
filtered_data.loc[:,'test']=['Hello', None, -7, 4.5, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]  
filtered_data.head()

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry,test
11831,133,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",Hello
23859,321,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",None
23860,321,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",-7
23861,321,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",4.5
23862,321,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",5


If there are some observed values only for some rows while the rest are missing / null:

First, the **None** value is assigned to all rows using `loc` method.

In [172]:
filtered_data.loc[:,'test'] = None

Then, the rows are updated with the observed values using `loc` method.

For updating the rows, the index of the rows is required.

The index of the rows can be identified either by printing the dataset or by using the `index` method.

In [173]:
print(filtered_data.index)

Index([11831, 23859, 23860, 23861, 23862, 23863, 23864, 24343, 24344, 24347,
       24444, 24445, 32673, 32674, 33881],
      dtype='int64')


In the following example, the index 11831, 23859, and 23862 are updated with their new values.

In [174]:
filtered_data.loc[[11831, 23859, 23862], 'test'] = [100, 200, '300']

In [175]:
filtered_data.head()

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry,test
11831,133,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",100
23859,321,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200
23860,321,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",None
23861,321,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",None
23862,321,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300


`````{admonition} Adding new attributes from another dataset!
:class: attention

In [Merging Chapter](https://stag.training.fdz.ioer.info/notebooks/309_merging_data.html#merging-based-on-common-attributes-using-merge) it has explained how to add more columns from another dataset.
`````

## Adding New Features (Rows)

To create a new feature (row) in the dataset, the `loc` method is used.

Here the index of the row is defined and for all the columns (CLC_st1, Biotpkt2018, ...) the values should be assigned.

In the following example, as the dataset includes 6 columns, 6 values should be defined for them.

In [177]:
filtered_data.loc['sample'] = ['Hello', 800, -7, 4.5, None, 9]
filtered_data.head(16)

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry,test
11831,133,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",100
23859,321,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200
23860,321,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",None
23861,321,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",None
23862,321,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300
23863,321,20.164493,1251.411517,28498.211647,"MULTIPOLYGON (((401726.203 5658676.434, 401724...",None
23864,321,20.164493,1055.657824,15517.253917,"MULTIPOLYGON (((403428.145 5659215.144, 403416...",None
24343,133,6.035751,287.703701,4278.338534,"MULTIPOLYGON (((411656.913 5656302.92, 411664....",None
24344,133,6.035751,183.544225,2066.507754,"MULTIPOLYGON (((411567.287 5656337.438, 411562...",None
24347,133,6.035751,213.015432,2684.440542,"MULTIPOLYGON (((411587.489 5655959.618, 411588...",None


`````{admonition} Adding new features from another dataset!
:class: attention

In [Merging Chapter](https://stag.training.fdz.ioer.info/notebooks/309_merging_data.html#merging-similar-datasets-using-concat) it has explained how to add more features from another dataset.
`````

## Copy a Column

Copying a column in a dataset is the same as adding a new column, except that instead of defining values, the values from the column of interest are imported.

In the example below, the values from the `test` column are copied into the `test_2` column. 

In [179]:
filtered_data.loc[:,'test_2'] = filtered_data.loc[:,'test']
filtered_data.head(17)

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry,test,test_2
11831,133,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",100,100
23859,321,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200,200
23860,321,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",None,None
23861,321,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",None,None
23862,321,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300,300
23863,321,20.164493,1251.411517,28498.211647,"MULTIPOLYGON (((401726.203 5658676.434, 401724...",None,None
23864,321,20.164493,1055.657824,15517.253917,"MULTIPOLYGON (((403428.145 5659215.144, 403416...",None,None
24343,133,6.035751,287.703701,4278.338534,"MULTIPOLYGON (((411656.913 5656302.92, 411664....",None,None
24344,133,6.035751,183.544225,2066.507754,"MULTIPOLYGON (((411567.287 5656337.438, 411562...",None,None
24347,133,6.035751,213.015432,2684.440542,"MULTIPOLYGON (((411587.489 5655959.618, 411588...",None,None


## Copy a Row 

Copying a row in a dataset is the same as adding a new row, except that instead of defining values, the values from the row of interest are imported.

In the example below, the values from the `sample` row are copied into the `sample_2` row. 

In [181]:
filtered_data.loc['sample_2'] = filtered_data.loc['sample']
filtered_data.head(18)

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry,test,test_2
11831,133,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",100,100
23859,321,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200,200
23860,321,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",None,None
23861,321,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",None,None
23862,321,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300,300
23863,321,20.164493,1251.411517,28498.211647,"MULTIPOLYGON (((401726.203 5658676.434, 401724...",None,None
23864,321,20.164493,1055.657824,15517.253917,"MULTIPOLYGON (((403428.145 5659215.144, 403416...",None,None
24343,133,6.035751,287.703701,4278.338534,"MULTIPOLYGON (((411656.913 5656302.92, 411664....",None,None
24344,133,6.035751,183.544225,2066.507754,"MULTIPOLYGON (((411567.287 5656337.438, 411562...",None,None
24347,133,6.035751,213.015432,2684.440542,"MULTIPOLYGON (((411587.489 5655959.618, 411588...",None,None


## Finding Null Values

To find the columns that include null values:
- `isnull`: Provide the same dataset but with the True (if it is a null value) or False value.
- `any`: Check if the columns include at least one True value.

In the following example, the columns `geometry`, `test` and `test_2` include **at least** one null (missing) values.

In [182]:
filtered_data.isnull().any()

CLC_st1         False
Biotpkt2018     False
Shape_Length    False
Shape_Area      False
geometry         True
test             True
test_2           True
dtype: bool

Then for the columns including null values, the related index is extracted.

In [183]:
filtered_data[filtered_data['test'].isnull()].index

Index([23860, 23861, 23863, 23864, 24343, 24344, 24347, 24444, 24445, 32673,
       32674, 33881],
      dtype='object')

Also it is possible to extract all the rows that include **at least** one null value.

- `isnull`: Provide the same dataset but with the True (if it is a null value) or False value.
- `any(axis=1)`: **axis = 1** checks for the True values in the **rows** of dataset. 

In [184]:
filtered_data[filtered_data.isnull().any(axis=1)]

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry,test,test_2
23860,321,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",None,None
23861,321,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",None,None
23863,321,20.164493,1251.411517,28498.211647,"MULTIPOLYGON (((401726.203 5658676.434, 401724...",None,None
23864,321,20.164493,1055.657824,15517.253917,"MULTIPOLYGON (((403428.145 5659215.144, 403416...",None,None
24343,133,6.035751,287.703701,4278.338534,"MULTIPOLYGON (((411656.913 5656302.92, 411664....",None,None
24344,133,6.035751,183.544225,2066.507754,"MULTIPOLYGON (((411567.287 5656337.438, 411562...",None,None
24347,133,6.035751,213.015432,2684.440542,"MULTIPOLYGON (((411587.489 5655959.618, 411588...",None,None
24444,411,17.079406,389.878192,3208.873688,"MULTIPOLYGON (((413977.74 5649257.41, 413976.8...",None,None
24445,411,17.079406,786.310517,17091.353830,"MULTIPOLYGON (((401901.386 5658342.569, 401914...",None,None
32673,321,20.164493,755.991947,15304.813349,"MULTIPOLYGON (((414609.258 5667182.283, 414600...",None,None


## Removing Null Values

To remove the rows that include `null` values in any columns, the `dropna` method is used.

In the following example, only 3 rows out of all rows do not include any null (missing) values in their columns. 

In [185]:
filtered_data.dropna()

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry,test,test_2
11831,133,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",100,100
23859,321,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200,200
23862,321,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300,300


To remove the rows that include `null` values in a specific column, the name of the column(s) are defined in the `subset` parameter of the `dropna` method.

In the following example, only 5 rows out of all rows do not include any null (missing) values in the `test` column. 

In [186]:
filtered_data.dropna(subset=['test'])

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry,test,test_2
11831,133,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",100,100
23859,321,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200,200
23862,321,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300,300
sample,Hello,800.000000,-7.000000,4.500000,None,9,9
sample_2,Hello,800.000000,-7.000000,4.500000,None,9,9


To remove columns with null values the `dropna` method is used with the `axis=1` parameter. 

In the following example, the columns `geometry`, `test` and `test_2` that include `null` values are removed from the dataset.

In [187]:
filtered_data.dropna(axis=1)

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area
11831,133,6.035751,17.828953,0.432276
23859,321,20.164493,618.702965,18317.038350
23860,321,20.164493,1099.972947,10894.472486
23861,321,20.164493,1023.775893,22918.685826
23862,321,20.164493,1126.678849,17980.357862
23863,321,20.164493,1251.411517,28498.211647
23864,321,20.164493,1055.657824,15517.253917
24343,133,6.035751,287.703701,4278.338534
24344,133,6.035751,183.544225,2066.507754
24347,133,6.035751,213.015432,2684.440542


## Removing Attributes (Columns)

To remove the extra columns from the dataset, the method `drop` is used. 

The name of the columns are defined in the method, and the parameter `axis` is set to 1.

In the example below, the columns `test_2` and `CLC_st1` are removed.

In [188]:
filtered_data.drop(['test_2','CLC_st1'], axis=1)

,Biotpkt2018,Shape_Length,Shape_Area,geometry,test
11831,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",100
23859,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200
23860,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",None
23861,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",None
23862,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300
23863,20.164493,1251.411517,28498.211647,"MULTIPOLYGON (((401726.203 5658676.434, 401724...",None
23864,20.164493,1055.657824,15517.253917,"MULTIPOLYGON (((403428.145 5659215.144, 403416...",None
24343,6.035751,287.703701,4278.338534,"MULTIPOLYGON (((411656.913 5656302.92, 411664....",None
24344,6.035751,183.544225,2066.507754,"MULTIPOLYGON (((411567.287 5656337.438, 411562...",None
24347,6.035751,213.015432,2684.440542,"MULTIPOLYGON (((411587.489 5655959.618, 411588...",None


## Removing Features (Rows)

To remove the extra rows from the dataset, the method `drop` is used and the index of the rows are defined to remove from the dataset.

In the example below, rows with the index of `11831` and `sample` are removed.

In [189]:
filtered_data.drop([11831,'sample'])

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry,test,test_2
23859,321,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200,200
23860,321,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",None,None
23861,321,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",None,None
23862,321,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300,300
23863,321,20.164493,1251.411517,28498.211647,"MULTIPOLYGON (((401726.203 5658676.434, 401724...",None,None
23864,321,20.164493,1055.657824,15517.253917,"MULTIPOLYGON (((403428.145 5659215.144, 403416...",None,None
24343,133,6.035751,287.703701,4278.338534,"MULTIPOLYGON (((411656.913 5656302.92, 411664....",None,None
24344,133,6.035751,183.544225,2066.507754,"MULTIPOLYGON (((411567.287 5656337.438, 411562...",None,None
24347,133,6.035751,213.015432,2684.440542,"MULTIPOLYGON (((411587.489 5655959.618, 411588...",None,None
24444,411,17.079406,389.878192,3208.873688,"MULTIPOLYGON (((413977.74 5649257.41, 413976.8...",None,None


## Filtering Data

It is also possible to explore the range of the data for numeric values, such as area.


To find the minimum and maximum values of the features, the following code is used:
- `min`: minimum value in the specified attribute
- `max`: maximum value in the specified attribute
- `:0.8f`: `:f` treated the value as a float, and `0.8` rounded the number of decimals to 8 numbers



`````{admonition} 
:class: tip
For more controlling over the output visit[here](https://python.aims.ac.za/pages/format_str.html#id4)
`````



In [190]:
min_value = gdf['Shape_Area'].min()
max_value = gdf['Shape_Area'].max()
print(f"minimum: {min_value:0.8f}")
print(f"maximum: {max_value:0.2f}")

minimum: 0.00000562
maximum: 3249895.12


Now that information about the attributes of features and the values assigned to them has been obtained, it is possible to decide which part of the data is of interest.


For example, it can be decided to work only with the features having areas less than 1000.


In [191]:
filter_db = gdf[gdf['Shape_Area'] < 1000]

In [192]:
filter_db.head()

,CLC_st1,Biotpkt2018,Shape_Length,Shape_Area,geometry
1,122,5.271487,31.935928,50.075513,"MULTIPOLYGON (((417850.525 5650376.33, 417846...."
3,122,5.271487,24.509066,36.443441,"MULTIPOLYGON (((423453.146 5650332.06, 423453...."
4,122,5.271487,29.937138,40.494155,"MULTIPOLYGON (((417331.434 5650889.039, 417330..."
6,122,5.271487,390.293053,764.349833,"MULTIPOLYGON (((407613.154 5651943.2, 407618.7..."
7,122,5.271487,34.931619,45.486260,"MULTIPOLYGON (((416066.508 5651463.834, 416066..."


In the chapter [Creating Maps](https://stag.training.fdz.ioer.info/notebooks/309_merging_data.html#merging-similar-datasets-using-concat), it is explained how to plot this data.